In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [ ]:
#QPU connection
try:
    from qat.qlmaas import QLMaaSConnection
    connection = QLMaaSConnection('qlm')
    LinAlg = connection.get_qpu("qat.qpus:LinAlg")
    lineal_qpu = LinAlg()
except (ImportError, OSError) as e:
    print('Problem: usin PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()

In [ ]:
%load_ext qat.core.magic

## 1. Quantum Multiplexors

Implementation of loading data routines using the *Lov Grover and Terry Rudolph* routines directly, using controlled rotations by state, is highly inneficient. In general the use of controlled rotations generate highly deep quantum circuits prone to errors. 
A more efficient approach is the use of Quantum Multiplexors where the only controlled gates are c-Not ones. 

The operation we want to implement is a controlled rotation on an angle $\theta_i$ controlled by a state $|i\rangle$. 

For a 2 qbits state the operator should be:

$$
\hat{U}|q_0q_1\rangle= \{ \begin{array}{ll}
      |0\rangle\hat{R}_y(\theta_0)|q_1\rangle  & if \;|q_0\rangle = |0\rangle \\
      |1\rangle\hat{R}_y(\theta_1)|q_1\rangle  & if \;|q_0\rangle = |1\rangle \\
\end{array}
$$

With a quantum multiplexor this operation can be implemented as:

$$\hat{U}(\theta_0, \theta_1)|q_0q_1\rangle= \left( \mathbf{I} \otimes \hat{R}_{y} (\frac{\theta_0+\theta_1}{2}) \right) \hat{C}_{NOT} \left(\mathbf{I} \otimes \hat{R}_{y} (\frac{\theta_0-\theta_1}{2}) \right) \hat{C}_{NOT}|q_0q_1\rangle$$

The circuit representation for this operator is:

![alt text](./QM_01.png)

As can be seen the quantum multiplexor needs only $C_{NOT}$ gates that are implemented in easier way than controlled rotations


For controlled state rotations of more angles quantum multiplexors can be implemented in a recursively way. For example if we want apply a controlled 4 angle rotation $[\theta_0, \theta_1, \theta_2, \theta_3]$ we can use following circuit:

![alt text](./QM_02.png)


In general a quantum multiplexor with $m$ control qubits can be decomposed as 2 multiplexors with $m − 1$ control qubits and 2 $\hat{C}_{NOT}$ gates:
![alt text](./QM_03.png)

In the **QuantumMultiplexors_Module** there are an implementation for a quantum multiplexor controlled by $m$ qbits and the functions neede to use them in order to load probability and a Integral function

## 2. Loading Data Gates: $\mathcal{P}$ and $\mathcal{R}$

First thing we need are the $\mathcal{P}$ and $\mathcal{R}$ for loading probability distribution and integral function respectively into a Quantum State.

In [ ]:
sys.path.append('/home/cesga/gferro/NEASQC/PhaseAmplification/')

In [ ]:
from AuxiliarFunctions import PostProcessResults, RunJob, get_histogram, TestBins, LeftConditionalProbability

In [ ]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

### 2.1 Loading Probability gate: $\mathcal{P}$

Following cell creates the AbstractGate and the function generator for the Loading Probability gate: $\mathcal{P}$. The **multiplexor_RY_m**  recursive function is needed in order to implement controlled rotations by state with quantum multiplexors.

In [ ]:
from qat.lang.AQASM import QRoutine, AbstractGate, RY
from QuantumMultiplexors_Module import  multiplexor_RY_m
def P_generatorQM(Dictionary):
    """
    Function generator for the AbstractGate that allows the loading of a discretized Probability
    in a Quantum State using Quantum Multiplexors
    Inputs:
        * ProbabilityArray: dict. Python dictionary whit a key named "array" whose corresponding item is a numpy array with the discretized
    probability to load. If ProbabilityArray = Dictionary['array']. The number of qbits will be log2(len(ProbabilityArray)). 
    Outuput:
        * qrout: Quantum routine. Routine for loading the discrete probability with Quantum Multiplexors.
    """
    
    
    ProbabilityArray = Dictionary['array']
    nqbits = TestBins(ProbabilityArray, text='Function')
    
    qrout = QRoutine()
    reg = qrout.new_wires(nqbits)
    # Now go iteratively trough each qubit computing the probabilities and adding the corresponding multiplexor
    for m in range(nqbits):
        #Calculates Conditional Probability
        ConditionalProbability = LeftConditionalProbability(m, ProbabilityArray)        
        #Rotation angles: length: 2^(i-1)-1 and i the number of qbits of the step
        thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))   
        
        if m == 0:
            # In the first iteration it is only needed a RY gate
            qrout.apply(RY(thetas[0]), reg[0])
        else:
            # In the following iterations we have to apply multiplexors controlled by m qubits
            # We call a function to construct the multiplexor, whose action is a block diagonal matrix of Ry gates with angles theta
            multiplexor_RY_m(qrout, reg, thetas, m, m)        
    return qrout  

LoadP_Gate = AbstractGate(
    "P_Gate",
    [dict],
    circuit_generator = P_generatorQM,
    arity = lambda x:TestBins(x['array'], 'Function')
)


In [ ]:
from qat.lang.AQASM import Program
qprog = Program()
qbits = qprog.qalloc(n_qbits)
P_gate = LoadP_Gate({'array':p_X})
qprog.apply(P_gate, qbits)
circuit = qprog.to_circ()

In [ ]:
%qatdisplay circuit --depth 0

In [ ]:
job = circuit.to_job()
result = lineal_qpu.submit(job)
P_results = PostProcessResults(RunJob(result))

In [ ]:
np.isclose(P_results['Probability'], p_X).all()

### 2.2 Loading Integral Function gate: $\mathcal{R}$

Following cell creates the AbstractGate and the function generator for the Loading Integral Function gate: $\mathcal{P}$. The **multiplexor_RY_m**  recursive function is needed in order to implement controlled rotations by state with quantum multiplexors.

In [ ]:
from qat.lang.AQASM import QRoutine, AbstractGate, RY
from QuantumMultiplexors_Module import  multiplexor_RY_m
def R_generatorQM(Dictionary):
    """
    Function generator for creating an AbstractGate that allows the loading of the integral of a given
    discretized function array into a Quantum State using Quantum Multiplexors
    Inputs:
        * Dictionary: dict. Python dictionary with a key named "array" whose corresponding item is a numpy array with the discrietized function. If the discretized function is FunctionArray = Dictionary['array'] the number of qbits will be log2(len(FunctionArray)) + 1 qbits.
    Outuput:
        * qrout: quantum routine. Routine for loading the input function as a integral on the last qbit using Quantum Multiplexors
    """
    
    FunctionArray = Dictionary['array']
    
    assert np.all(FunctionArray<=1.), 'The image of the function must be less than 1. Rescaling is required'
    assert np.all(FunctionArray>=0.), 'The image of the function must be greater than 0. Rescaling is required'
    assert isinstance(FunctionArray, np.ndarray), 'the output of the function p must be a numpy array'    
    
    nqbits = TestBins(FunctionArray, text='Function')
    #Calculation of the rotation angles
    thetas = 2.0*np.arcsin(np.sqrt(FunctionArray))
    

    qrout = QRoutine()
    reg = qrout.new_wires(nqbits+1)
    multiplexor_RY_m(qrout, reg, thetas, nqbits, nqbits)
    return qrout    

LoadR_Gate = AbstractGate(
    "R_Gate",
    [dict],
    circuit_generator = R_generatorQM,
    arity = lambda x:TestBins(x['array'], 'Function')+1
)


In [ ]:
from qat.lang.AQASM import Program, H
qprog = Program()
qbits = qprog.qalloc(n_qbits+1)
for i in range(n_qbits):
    qprog.apply(H, qbits[i])
R_gate = LoadR_Gate({'array':f_X})
qprog.apply(R_gate, qbits)
circuit = qprog.to_circ()

In [ ]:
%qatdisplay circuit --depth 0

In [ ]:
job = circuit.to_job(qubits = [n_qbits])
result = lineal_qpu.submit(job)
R_results = PostProcessResults(RunJob(result))

In [ ]:
#Integral of f(x)
MeasurementIntegral = R_results['Probability'][1]*2**(n_qbits)
np.isclose(MeasurementIntegral, sum(f_X))